In [10]:
import pandas as pd
import httpx
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import plotly.express as px
import sweetviz as sv
import seaborn as sns
import causalpy as cp
import sklearn as skl
import statsmodels.tsa.filters.hp_filter as smf
import matplotlib.dates as mdates

from sklearn.linear_model import LinearRegression
from sklearn.e
from ipyleaflet import Map, Marker, basemaps, LayersControl


ImportError: cannot import name 'ExpornentialRegression' from 'sklearn.linear_model' (/Users/sofiepelvig/apartments/.pixi/env/lib/python3.10/site-packages/sklearn/linear_model/__init__.py)

In [2]:
df = pd.read_csv("/Users/sofiepelvig/apartments/assets/data/bolig.csv.gzip", compression="gzip", parse_dates=["soldDate"])
df

,estateId,address,zipCode,price,soldDate,propertyType,saleType,sqmPrice,rooms,size,...,change,guid,latitude,longitude,municipalityCode,estateCode,city,groupKey,canGetVR,bfEnr
0,0,"Åboulevard 35, 1. th",1960,5400000,2023-09-20 22:00:00+00:00,3,Alm. Salg,55102.040,4.0,98,...,0.000000,1E65440F-F71C-4DC0-9B4F-CFF6782EB4D6,55.684840,12.551123,147,221223,Frederiksberg C,NaN,True,184708
1,0,"Borups Alle 116, 2. th",2000,4195000,2023-09-14 22:00:00+00:00,3,Alm. Salg,52437.500,3.0,80,...,0.000000,7F866136-B56E-4BE5-8695-0E6DAD2E00B0,55.693672,12.533304,147,13582,Frederiksberg,NaN,True,175058
2,1993747,"Harsdorffsvej 4B, st. th",1874,3475000,2023-09-13 22:00:00+00:00,3,Alm. Salg,56967.215,2.0,61,...,-0.572246,6310B883-5F91-4690-BDDB-15157487458E,55.679310,12.546504,147,188935,Frederiksberg C,NaN,True,183893
3,0,"Roskildevej 53, 2. 207",2000,1250000,2023-09-13 22:00:00+00:00,3,Fam. Salg,19230.770,2.0,65,...,0.000000,583219EB-22E7-4D53-9BF0-D8786F3953CD,55.671980,12.511993,147,175566,Frederiksberg,NaN,True,184459
4,1999954,"Rådmand Steins Alle 15, 2. th",2000,3744000,2023-09-12 22:00:00+00:00,3,Alm. Salg,34990.652,4.0,107,...,0.000000,191C546B-C71B-4C89-8AED-2967AFC138C7,55.672897,12.499296,147,108524,Frederiksberg,NaN,True,186808
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99206,0,"Hørsholmsgade 8, 3. tv",2200,11947874,1992-01-06 23:00:00+00:00,3,Alm. Salg,225431.580,2.0,53,...,0.000000,805A8D01-9BF6-4047-8164-4DF8324C489D,55.690900,12.543945,101,260348,København N,1.0,True,6019808
99207,0,"Hørsholmsgade 8, 4. th",2200,11947874,1992-01-06 23:00:00+00:00,3,Alm. Salg,217234.080,2.0,55,...,0.000000,3243A2DA-3BAE-493A-BFC7-F6ACAE031022,55.690900,12.543945,101,260348,København N,1.0,True,6019808
99208,0,"Hørsholmsgade 8, 4. tv",2200,11947874,1992-01-06 23:00:00+00:00,3,Alm. Salg,225431.580,2.0,53,...,0.000000,DA7226E2-F5C9-43D1-AEE6-F928CE45212E,55.690900,12.543945,101,260348,København N,1.0,True,6019808
99209,0,"Peter Fabers Gade 41, 2. th",2200,250000,1992-01-03 23:00:00+00:00,3,Alm. Salg,5000.000,2.0,50,...,0.000000,D9388837-4672-4B9A-A1FE-C65BB6397DD8,55.691830,12.556379,101,436128,København N,NaN,True,146895


In [4]:
#Sorting fam_salg away
alm_salg_df = df[df['saleType'] == 'Alm. Salg']

#Sorting apartments away that where sold for under 500.000 and above 9.000.000
alm_salg_df = alm_salg_df[(alm_salg_df['price'] > 5e5) & (alm_salg_df['price'] < 9e6)]

#Sorting apartments away that have above 5 roomes doe to the likelyhood of it being under 0.001%procent
alm_salg_df = alm_salg_df[(alm_salg_df['rooms'] < 6 )]

alm_salg_df = alm_salg_df.sort_values(by="soldDate")
alm_salg_df

,estateId,address,zipCode,price,soldDate,propertyType,saleType,sqmPrice,rooms,size,...,change,guid,latitude,longitude,municipalityCode,estateCode,city,groupKey,canGetVR,bfEnr
17251,0,"Erik Menveds Vej 4, 3. tv",1965,800000,1992-01-01 23:00:00+00:00,3,Alm. Salg,8247.4230,5.0,97,...,0.000000,E3200C3D-DAF2-4913-8B4D-E64EF0415B28,55.683414,12.548503,147,29195,Frederiksberg C,1.0,True,100026395
17250,0,"Erik Menveds Vej 4, 2. th",1965,800000,1992-01-01 23:00:00+00:00,3,Alm. Salg,7339.4497,5.0,109,...,0.000000,1221F901-DEAF-4054-85E4-6946CA083656,55.683414,12.548503,147,29195,Frederiksberg C,1.0,True,100026395
17249,0,"Erik Menveds Vej 4, 2. tv",1965,800000,1992-01-01 23:00:00+00:00,3,Alm. Salg,7619.0480,4.0,105,...,0.000000,D7EFBD63-4F53-43D0-9098-C4C6495A2E0D,55.683414,12.548503,147,29195,Frederiksberg C,4.0,True,100026395
17248,0,"Erik Menveds Vej 4, st. tv",1965,800000,1992-01-01 23:00:00+00:00,3,Alm. Salg,7619.0480,5.0,105,...,0.000000,D665049D-BC24-42E4-BF9D-2A01DC3EDD18,55.683414,12.548503,147,29195,Frederiksberg C,4.0,True,100026395
17247,0,"Erik Menveds Vej 4, 1. th",1965,800000,1992-01-01 23:00:00+00:00,3,Alm. Salg,7339.4497,5.0,109,...,0.000000,E7F9AC4C-846F-4E2A-8AFE-248272DADAD9,55.683414,12.548503,147,29195,Frederiksberg C,4.0,True,100026395
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17259,0,"Trekronergade 68, 1",2500,3600000,2023-09-20 22:00:00+00:00,3,Alm. Salg,48000.0000,3.0,75,...,0.000000,4A88D732-0155-417F-8CDC-C6CD01437919,55.655933,12.519941,101,792463,Valby,NaN,True,170165
17257,1981970,"Strandlodsvej 19C, 7",2300,7800000,2023-09-21 22:00:00+00:00,3,Alm. Salg,60937.5000,5.0,128,...,-2.439024,B697A6B1-15F4-499B-852E-63FF0AC98F59,55.665154,12.623071,101,24984,København S,NaN,True,130834
17256,2009076,"Klaksvigsgade 12, 1. th",2300,4725000,2023-09-21 22:00:00+00:00,3,Alm. Salg,50806.4530,2.0,93,...,-2.476780,669B6DC1-64D6-4C8B-9CF7-588EA7E6997A,55.667297,12.580413,101,1078,København S,NaN,True,104220
17254,0,"Holmbladsgade 58, 3. th",2300,3490000,2023-09-21 22:00:00+00:00,3,Alm. Salg,45324.6760,3.0,77,...,0.000000,62FF5AEA-4326-4E56-92E4-751BFACD94CB,55.665596,12.609646,101,233774,København S,NaN,True,123837


In [5]:
(alm_salg_df
  .sample(5)
  .loc[:, ["price", "rooms", "size", "soldDate"]]
  .assign(soldAge = lambda d: (d["soldDate"].dt.date - pd.to_datetime("1991-01-01")).dt.days/365)
  .loc[:,["price", "rooms", "size", "soldAge"]] 
)

TypeError: unsupported operand type(s) for -: 'numpy.ndarray' and 'Timestamp'

In [6]:
(alm_salg_df["soldDate"] - pd.to_datetime("1991-01-01", utc=True)).dt.days/365

17251     1.000000
17250     1.000000
17249     1.000000
17248     1.000000
17247     1.000000
           ...    
17259    32.739726
17257    32.742466
17256    32.742466
17254    32.742466
17255    32.742466
Name: soldDate, Length: 68104, dtype: float64

In [7]:
train_data = (alm_salg_df
  .loc[:, ["price", "rooms", "size", "soldDate"]]
  .assign(soldAge = lambda d: (d["soldDate"] - pd.to_datetime("1991-12-31", utc=True)).dt.days/365)
  .loc[:,["price", "rooms", "size", "soldAge"]]
  .sort_values(by="soldAge")
)

In [8]:
# price ~ rooms + size + (year - 1 jan 1992)

X = train_data[["rooms", "size", "soldAge"]].map(np.log)
y = train_data["price"].map(np.log)



In [9]:
reg = ExpornentialRegression()
reg.fit(X,y)
np.exp(reg.coef_[0])

NameError: name 'ExpornentialRegression' is not defined

In [ ]:
coefs = np.exp(reg.coef_)
print("Multiplicative effects on price:")
print(f"Number of rooms: {coefs[0]}")
print(f"Size: {coefs[1]}")
print(f"Sold age: {coefs[2]}")

In [ ]:
from scipy import stats

z_scores = np.abs(stats.zscore(train_data))
outliers = np.where(z_scores > 2)  # Adjust the threshold as needed
outliers

In [ ]:
np.exp(reg.intercept_)

In [ ]:
y_hat = reg.predict(X)

In [ ]:
(pd
     .DataFrame({"price": np.exp(y), "y_pred": np.exp(y_hat).round(0)})
     .assign(residuals = lambda d: abs(d["price"] - d["y_pred"]),
             weird = lambda d: d["y_pred"] / d["y_pred"].max())
     .nlargest(5, "residuals")
     .loc[lambda d: d["weird"] < 1]
    
    )

In [ ]:
g = (pd
     .DataFrame({"price": np.exp(y), "y_pred": np.exp(y_hat).round(0)})
     .assign(residuals = lambda d: abs(d["price"] - d["y_pred"]),
             weird = lambda d: d["y_pred"] / d["y_pred"].max())
     #.nlargest(5, "residuals")
     .loc[lambda d: d["weird"] < 1]
    
    )
g

In [ ]:
g = g.reset_index(drop=True)
g.max()

In [ ]:
plt.plot(g.index, g.sort_values(by="y_pred")["y_pred"], c='b')
plt.scatter(g.index,g['price'],alpha=.1, s=3)
plt.ylabel('Price dkk in mil')
plt.xlabel('Dayes past since start date 01.01.1991')
plt.title('Linear Regression, predicting price over time vs. acutal price over time')
plt.ticklabel_format(axis='y', style='plain')
plt.show()